# 지하철 1호선 복합 지표 분석
## 소요시간 × 배차 간격 = 승객 대기 시간 지표

이 분석은 **역간 소요시간**과 **배차 간격**을 결합하여 승객의 실제 경험을 정량화합니다.

### 핵심 개념
```
복합 지표 = 평균 소요시간 × 평균 배차 간격
```

**의미**:
- 소요시간이 길다 → 이동에 오래 걸림
- 배차 간격이 길다 → 열차 대기 시간이 김
- 둘 다 크면 → 승객 불편이 큼 (혼잡도 높을 가능성)

**활용**:
- 서비스 개선이 필요한 구간 식별
- 배차 증편이 필요한 구간 우선순위 결정
- 혼잡도 예측 모델의 입력 변수

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sqlite3
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# DB 경로 및 테이블 설정
DB_PATH = r"..\input\data\metro_datetime.db"
TABLE = "Metro_Line_1_Schedule"

print("✅ 라이브러리 임포트 완료")

## 1. 전체 데이터 로드 및 전처리

In [ ]:
# 데이터 로드
with sqlite3.connect(DB_PATH) as conn:
    query = f"""
        SELECT 
            `역사코드`,
            `역사명`,
            `주중주말`,
            `방향`,
            `급행여부`,
            `열차코드`,
            `열차도착시간_dt`,
            `열차출발시간_dt`,
            `출발역`,
            `도착역`
        FROM {TABLE}
        WHERE `급행여부` = 0
        ORDER BY `열차코드`, `열차도착시간_dt`
    """
    df_all = pd.read_sql(query, conn)

# datetime 변환
df_all['열차도착시간_dt'] = pd.to_datetime(df_all['열차도착시간_dt'])
df_all['열차출발시간_dt'] = pd.to_datetime(df_all['열차출발시간_dt'])

print(f"📊 전체 데이터: {len(df_all):,}행")
print(f"🚇 고유 열차 수: {df_all['열차코드'].nunique():,}대")
print(f"🚉 고유 역 수: {df_all['역사명'].nunique():,}개")
print(f"📅 주중주말: {df_all['주중주말'].unique()}")

display(df_all.head())

## 2. 역간 소요시간 계산

각 열차의 연속된 정차역 사이의 이동 시간을 계산합니다.

In [ ]:
def calculate_inter_station_time(df):
    """
    역간 소요시간 계산
    """
    travel_times = []
    
    for (train_code, day_type), group in df.groupby(['열차코드', '주중주말']):
        group = group.sort_values('열차도착시간_dt').reset_index(drop=True)
        
        for i in range(len(group) - 1):
            current = group.iloc[i]
            next_station = group.iloc[i + 1]
            
            if pd.notna(current['열차출발시간_dt']) and pd.notna(next_station['열차도착시간_dt']):
                travel_time = (next_station['열차도착시간_dt'] - current['열차출발시간_dt']).total_seconds() / 60
                
                if 0 < travel_time < 60:
                    travel_times.append({
                        '출발역명': current['역사명'],
                        '도착역명': next_station['역사명'],
                        '출발역코드': current['역사코드'],
                        '도착역코드': next_station['역사코드'],
                        '소요시간_분': travel_time,
                        '열차코드': train_code,
                        '주중주말': day_type,
                        '방향': current['방향']
                    })
    
    return pd.DataFrame(travel_times)

print("⏱️ 역간 소요시간 계산 중...")
df_travel = calculate_inter_station_time(df_all)

# 구간별 평균 소요시간
travel_stats = df_travel.groupby(['출발역명', '도착역명', '방향', '주중주말']).agg({
    '소요시간_분': ['mean', 'std', 'count']
}).round(2)
travel_stats.columns = ['평균소요시간_분', '표준편차', '측정횟수']
travel_stats = travel_stats.reset_index()

print(f"✅ {len(df_travel):,}개의 역간 이동 기록")
print(f"✅ {len(travel_stats):,}개의 고유 구간 (방향별, 주중주말별)")

display(travel_stats.head(20))

## 3. 배차 간격(Headway) 계산

각 역에서 연속된 열차가 도착하는 시간 간격을 계산합니다.

In [ ]:
def calculate_headway_by_station(df):
    """
    역별, 방향별, 주중주말별 배차 간격 계산
    """
    headways = []
    
    for (station, direction, day_type), group in df.groupby(['역사명', '방향', '주중주말']):
        # 도착시간이 있는 데이터만
        group = group[group['열차도착시간_dt'].notna()].copy()
        group = group.sort_values('열차도착시간_dt').reset_index(drop=True)
        
        if len(group) > 1:
            # 배차 간격 계산
            group['배차간격_분'] = group['열차도착시간_dt'].diff().dt.total_seconds() / 60
            
            # 첫 번째 행 제외 (diff로 인한 NaN)
            valid_headways = group['배차간격_분'].dropna()
            
            # 이상치 제거 (1분~60분 범위)
            valid_headways = valid_headways[(valid_headways > 1) & (valid_headways < 60)]
            
            if len(valid_headways) > 0:
                headways.append({
                    '역사명': station,
                    '역사코드': group.iloc[0]['역사코드'],
                    '방향': direction,
                    '주중주말': day_type,
                    '평균배차간격_분': valid_headways.mean(),
                    '최소배차간격_분': valid_headways.min(),
                    '최대배차간격_분': valid_headways.max(),
                    '표준편차': valid_headways.std(),
                    '측정횟수': len(valid_headways)
                })
    
    return pd.DataFrame(headways)

print("⏱️ 배차 간격 계산 중...")
df_headway = calculate_headway_by_station(df_all)

print(f"\n✅ {len(df_headway):,}개의 역-방향-주중주말 조합")
print(f"\n평균 배차 간격: {df_headway['평균배차간격_분'].mean():.2f}분")

display(df_headway.head(20))

## 4. 복합 지표 계산: 소요시간 × 배차 간격

각 구간의 소요시간과 출발역의 배차 간격을 곱하여 복합 지표를 생성합니다.

In [ ]:
# 역간 소요시간에 출발역의 배차 간격 결합
df_combined = travel_stats.merge(
    df_headway[['역사명', '방향', '주중주말', '평균배차간격_분']],
    left_on=['출발역명', '방향', '주중주말'],
    right_on=['역사명', '방향', '주중주말'],
    how='left'
)

# 불필요한 컬럼 제거
df_combined = df_combined.drop('역사명', axis=1)

# 복합 지표 계산
df_combined['복합지표'] = df_combined['평균소요시간_분'] * df_combined['평균배차간격_분']

# 구간명 생성
df_combined['구간'] = df_combined['출발역명'] + ' → ' + df_combined['도착역명']

# 결측값 제거 (배차 간격 데이터가 없는 경우)
df_combined = df_combined.dropna(subset=['평균배차간격_분', '복합지표'])

print(f"✅ 복합 지표 계산 완료: {len(df_combined):,}개 구간")
print(f"\n복합 지표 통계:")
print(f"  평균: {df_combined['복합지표'].mean():.2f}")
print(f"  중앙값: {df_combined['복합지표'].median():.2f}")
print(f"  최소: {df_combined['복합지표'].min():.2f}")
print(f"  최대: {df_combined['복합지표'].max():.2f}")

display(df_combined.head(20))

## 5. 복합 지표 Top 50 분석 (주중)

승객 불편이 가장 큰 구간을 식별합니다.

In [ ]:
# 주중 데이터만 필터링
df_weekday = df_combined[df_combined['주중주말'] == 'DAY'].copy()

# 복합 지표 큰 순서로 정렬
df_weekday_sorted = df_weekday.sort_values('복합지표', ascending=False)

print("🚨 복합 지표 Top 50 (주중) - 승객 대기 시간이 긴 구간")
print("=" * 120)
print("복합지표가 크다 = 소요시간도 길고 + 배차 간격도 길다 = 승객 불편이 큼")
print("=" * 120)

# Top 50 표시
top50_weekday = df_weekday_sorted.head(50)[[
    '구간', '방향', '평균소요시간_분', '평균배차간격_분', '복합지표', '측정횟수'
]].reset_index(drop=True)

display(top50_weekday)

# CSV 저장
top50_weekday.to_csv('복합지표_Top50_주중.csv', index=False, encoding='utf-8-sig')
print("\n✅ '복합지표_Top50_주중.csv' 파일로 저장되었습니다.")

## 6. 복합 지표 Top 50 분석 (주말)

In [ ]:
# 주말 데이터
df_weekend = df_combined[df_combined['주중주말'] == 'SAT'].copy()
df_weekend_sorted = df_weekend.sort_values('복합지표', ascending=False)

print("🚨 복합 지표 Top 50 (주말)")
print("=" * 120)

top50_weekend = df_weekend_sorted.head(50)[[
    '구간', '방향', '평균소요시간_분', '평균배차간격_분', '복합지표', '측정횟수'
]].reset_index(drop=True)

display(top50_weekend)

# CSV 저장
top50_weekend.to_csv('복합지표_Top50_주말.csv', index=False, encoding='utf-8-sig')
print("\n✅ '복합지표_Top50_주말.csv' 파일로 저장되었습니다.")

## 7. 주중/주말 비교 분석

In [ ]:
# 주중/주말 비교를 위한 피벗
comparison = df_combined.pivot_table(
    index=['구간', '방향'],
    columns='주중주말',
    values=['평균소요시간_분', '평균배차간격_분', '복합지표'],
    aggfunc='mean'
).round(2)

# 컬럼 flatten
comparison.columns = ['_'.join(col).strip() for col in comparison.columns.values]
comparison = comparison.reset_index()

# 주중-주말 차이 계산
if '복합지표_DAY' in comparison.columns and '복합지표_SAT' in comparison.columns:
    comparison['복합지표_차이(주중-주말)'] = comparison['복합지표_DAY'] - comparison['복합지표_SAT']
    
    # 차이가 큰 순서로 정렬 (주중이 더 나쁜 구간)
    comparison_sorted = comparison.sort_values('복합지표_차이(주중-주말)', ascending=False)
    
    print("📊 주중/주말 복합 지표 차이 Top 30 (주중이 더 나쁜 구간)")
    print("=" * 120)
    print("양수 = 주중이 더 나쁨 (소요시간↑ or 배차간격↑)")
    print("음수 = 주말이 더 나쁨")
    print("=" * 120)
    
    display(comparison_sorted.head(30))
    
    # CSV 저장
    comparison_sorted.to_csv('복합지표_주중주말비교.csv', index=False, encoding='utf-8-sig')
    print("\n✅ '복합지표_주중주말비교.csv' 파일로 저장되었습니다.")

## 8. 시각화: 복합 지표 Top 30 (주중)

In [ ]:
# Top 30 구간
top30_viz = df_weekday_sorted.head(30).copy()

fig = px.bar(
    top30_viz,
    x='복합지표',
    y='구간',
    orientation='h',
    title='복합 지표 Top 30 (주중)<br><sub>소요시간 × 배차간격 = 승객 대기 시간</sub>',
    labels={'복합지표': '복합 지표 (분²)', '구간': '역 구간'},
    color='복합지표',
    color_continuous_scale='Reds',
    text='복합지표',
    hover_data={'평균소요시간_분': ':.2f', '평균배차간격_분': ':.2f'}
)

fig.update_traces(
    texttemplate='%{text:.1f}',
    textposition='outside'
)

fig.update_layout(
    height=900,
    yaxis={'categoryorder': 'total ascending'},
    showlegend=False
)

fig.show()

## 9. 시각화: 소요시간 vs 배차간격 산점도

In [ ]:
# 주중 데이터로 산점도
fig = px.scatter(
    df_weekday,
    x='평균소요시간_분',
    y='평균배차간격_분',
    size='복합지표',
    color='복합지표',
    hover_data=['구간', '방향'],
    title='소요시간 vs 배차간격 (주중)<br><sub>크기와 색 = 복합지표</sub>',
    labels={
        '평균소요시간_분': '평균 소요시간 (분)',
        '평균배차간격_분': '평균 배차간격 (분)'
    },
    color_continuous_scale='Reds'
)

# 사분면 표시
median_travel = df_weekday['평균소요시간_분'].median()
median_headway = df_weekday['평균배차간격_분'].median()

fig.add_hline(y=median_headway, line_dash="dash", line_color="gray", opacity=0.5)
fig.add_vline(x=median_travel, line_dash="dash", line_color="gray", opacity=0.5)

# 사분면 주석
fig.add_annotation(
    x=df_weekday['평균소요시간_분'].max() * 0.9,
    y=df_weekday['평균배차간격_분'].max() * 0.9,
    text="⚠️ 최악<br>(길다+드물다)",
    showarrow=False,
    bgcolor="rgba(255,0,0,0.1)"
)

fig.add_annotation(
    x=df_weekday['평균소요시간_분'].min() * 1.5,
    y=df_weekday['평균배차간격_분'].min() * 1.5,
    text="✅ 최적<br>(짧다+자주)",
    showarrow=False,
    bgcolor="rgba(0,255,0,0.1)"
)

fig.show()

print(f"\n📊 사분면 분석:")
print(f"  중앙값 - 소요시간: {median_travel:.2f}분")
print(f"  중앙값 - 배차간격: {median_headway:.2f}분")
print(f"\n  우상단(최악): 소요시간↑ + 배차간격↑")
print(f"  좌하단(최적): 소요시간↓ + 배차간격↓")

## 10. 시각화: 복합 지표 분포

In [ ]:
# 주중/주말 복합 지표 분포 비교
fig = px.histogram(
    df_combined,
    x='복합지표',
    color='주중주말',
    nbins=50,
    title='복합 지표 분포 (주중 vs 주말)',
    labels={'복합지표': '복합 지표 (분²)', 'count': '빈도'},
    barmode='overlay',
    opacity=0.7
)

# 평균선 추가
for day_type, color in [('DAY', 'blue'), ('SAT', 'red')]:
    if day_type in df_combined['주중주말'].values:
        mean_val = df_combined[df_combined['주중주말'] == day_type]['복합지표'].mean()
        fig.add_vline(
            x=mean_val,
            line_dash="dash",
            line_color=color,
            annotation_text=f"{day_type} 평균: {mean_val:.1f}",
            annotation_position="top"
        )

fig.show()

print("\n📊 복합 지표 통계:")
display(df_combined.groupby('주중주말')['복합지표'].describe().round(2))

## 11. 방향별 분석 (상행 vs 하행)

In [ ]:
# 방향별 복합 지표 비교 (주중)
direction_stats = df_weekday.groupby('방향').agg({
    '평균소요시간_분': 'mean',
    '평균배차간격_분': 'mean',
    '복합지표': 'mean',
    '구간': 'count'
}).round(2)
direction_stats.columns = ['평균_소요시간', '평균_배차간격', '평균_복합지표', '구간수']

print("🔀 방향별 복합 지표 비교 (주중):")
display(direction_stats)

# 방향별 Top 10
print("\n⬆️ 상행(UP) Top 10:")
up_top10 = df_weekday[df_weekday['방향'] == 'UP'].nlargest(10, '복합지표')[[
    '구간', '평균소요시간_분', '평균배차간격_분', '복합지표'
]]
display(up_top10)

print("\n⬇️ 하행(DOWN) Top 10:")
down_top10 = df_weekday[df_weekday['방향'] == 'DOWN'].nlargest(10, '복합지표')[[
    '구간', '평균소요시간_분', '평균배차간격_분', '복합지표'
]]
display(down_top10)

## 12. 시간대별 분석 (시간대별 배차 간격 변화)

In [ ]:
# 시간대별 배차 간격 계산을 위해 원본 데이터 사용
df_time = df_all[df_all['열차도착시간_dt'].notna()].copy()
df_time['시간대'] = df_time['열차도착시간_dt'].dt.hour

# 주요 역의 시간대별 배차 간격
major_stations = ['서울역', '영등포', '수원', '청량리']

for station in major_stations:
    if station in df_time['역사명'].values:
        station_data = df_time[
            (df_time['역사명'] == station) & 
            (df_time['주중주말'] == 'DAY')
        ].copy()
        
        if len(station_data) > 0:
            # 방향별로 그룹화
            hourly_headway = []
            
            for direction, group in station_data.groupby('방향'):
                group = group.sort_values('열차도착시간_dt')
                group['배차간격'] = group['열차도착시간_dt'].diff().dt.total_seconds() / 60
                
                hourly = group.groupby('시간대')['배차간격'].mean().reset_index()
                hourly['방향'] = direction
                hourly_headway.append(hourly)
            
            if hourly_headway:
                hourly_df = pd.concat(hourly_headway)
                
                fig = px.line(
                    hourly_df,
                    x='시간대',
                    y='배차간격',
                    color='방향',
                    title=f'{station} 시간대별 평균 배차간격 (주중)',
                    labels={'시간대': '시간 (시)', '배차간격': '평균 배차간격 (분)'},
                    markers=True
                )
                
                # 출퇴근 시간대 강조
                fig.add_vrect(x0=7, x1=9, fillcolor="red", opacity=0.1, 
                            annotation_text="출근", annotation_position="top left")
                fig.add_vrect(x0=18, x1=20, fillcolor="blue", opacity=0.1, 
                            annotation_text="퇴근", annotation_position="top left")
                
                fig.show()

## 13. 개선 우선순위 구간 선정

복합 지표가 높은 구간을 개선 대상으로 선정합니다.

In [ ]:
# 개선 우선순위 점수 계산
df_priority = df_weekday.copy()

# 정규화 (0~100 스케일)
df_priority['소요시간_점수'] = (df_priority['평균소요시간_분'] - df_priority['평균소요시간_분'].min()) / \
                            (df_priority['평균소요시간_분'].max() - df_priority['평균소요시간_분'].min()) * 100

df_priority['배차간격_점수'] = (df_priority['평균배차간격_분'] - df_priority['평균배차간격_분'].min()) / \
                            (df_priority['평균배차간격_분'].max() - df_priority['평균배차간격_분'].min()) * 100

df_priority['복합지표_점수'] = (df_priority['복합지표'] - df_priority['복합지표'].min()) / \
                            (df_priority['복합지표'].max() - df_priority['복합지표'].min()) * 100

# 종합 점수 (가중 평균)
df_priority['종합점수'] = (
    df_priority['소요시간_점수'] * 0.3 +
    df_priority['배차간격_점수'] * 0.3 +
    df_priority['복합지표_점수'] * 0.4
).round(2)

# 개선 방안 추천
def recommend_improvement(row):
    if row['배차간격_점수'] > row['소요시간_점수']:
        return '배차 증편 우선'
    elif row['소요시간_점수'] > row['배차간격_점수']:
        return '선로/신호 개선 우선'
    else:
        return '종합 개선 필요'

df_priority['개선방안'] = df_priority.apply(recommend_improvement, axis=1)

# Top 30 개선 우선순위
priority_list = df_priority.nlargest(30, '종합점수')[[
    '구간', '방향', '평균소요시간_분', '평균배차간격_분', '복합지표',
    '소요시간_점수', '배차간격_점수', '종합점수', '개선방안'
]].reset_index(drop=True)

print("🎯 개선 우선순위 Top 30 (주중)")
print("=" * 120)
print("종합점수 = 소요시간(30%) + 배차간격(30%) + 복합지표(40%)")
print("=" * 120)
display(priority_list)

# CSV 저장
priority_list.to_csv('개선_우선순위_Top30.csv', index=False, encoding='utf-8-sig')
print("\n✅ '개선_우선순위_Top30.csv' 파일로 저장되었습니다.")

# 개선 방안별 통계
print("\n📊 개선 방안별 구간 수:")
display(priority_list['개선방안'].value_counts())

## 14. 전체 데이터 저장

In [ ]:
# 전체 복합 지표 데이터 저장
df_combined_sorted = df_combined.sort_values('복합지표', ascending=False)

df_combined_sorted[[
    '구간', '방향', '주중주말',
    '평균소요시간_분', '표준편차', '평균배차간격_분', '복합지표', '측정횟수'
]].to_csv('복합지표_전체데이터.csv', index=False, encoding='utf-8-sig')

print("✅ 모든 분석 결과가 CSV 파일로 저장되었습니다.")
print("\n📁 생성된 파일 목록:")
print("  1. 복합지표_Top50_주중.csv")
print("  2. 복합지표_Top50_주말.csv")
print("  3. 복합지표_주중주말비교.csv")
print("  4. 개선_우선순위_Top30.csv")
print("  5. 복합지표_전체데이터.csv")

## 💡 분석 결론 및 인사이트

### 🔍 주요 발견사항

#### 1. 복합 지표의 의미
- **높은 복합 지표** = 소요시간 길다 + 배차 간격 길다
- 승객 입장: 열차를 오래 기다리고 + 이동도 오래 걸림
- **혼잡도 증가 가능성**: 대기 승객 누적 → 탑승 시 혼잡

#### 2. 사분면 분석
```
배차간격 ↑
    │
    │  ⚠️ 최악      │  배차↑소요↓
    │  (증편필요)   │
────┼──────────────┼──────── 소요시간 →
    │  배차↓소요↑   │  ✅ 최적
    │              │  (현상유지)
```

#### 3. 개선 우선순위
- **배차 증편 우선**: 배차 간격이 더 문제인 구간
- **선로/신호 개선**: 소요시간이 더 문제인 구간
- **종합 개선**: 둘 다 문제인 구간

#### 4. 시간대별 패턴
- 출퇴근 시간대: 배차 간격 감소 (증편 운행)
- 심야 시간대: 배차 간격 증가
- 소요시간은 상대적으로 일정

### 🎯 다음 분석 방향

#### 1. 실제 혼잡도와 연계
```python
복합지표 vs 실제혼잡도
→ 상관관계 분석
→ 예측 모델 구축
```

#### 2. 차량 노후도 연동
```python
노후 차량 → 소요시간↑?
복합지표 높은 구간에 노후 차량 많은가?
```

#### 3. 승하차 인원 데이터 결합
```python
복합지표 높은 구간 = 승객 많은 구간?
수요 대비 공급 분석
```

#### 4. 시뮬레이션
```python
만약 배차를 20% 증편하면?
만약 소요시간을 10% 단축하면?
→ 효과 예측
```

### 📊 정책 제안

1. **즉시 조치**: Top 10 구간 배차 증편
2. **중기 계획**: Top 30 구간 선로/신호 개선
3. **장기 전략**: 노선 재설계 검토

---

**이 분석은 데이터 기반 의사결정의 첫 단계입니다.**  
실제 정책 수립 시에는 예산, 기술적 제약, 이해관계자 의견 등을 종합적으로 고려해야 합니다.